<font size="6"><b>MLP Neural Networks - Part IV</b></font>

# Libraries

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xgboost as xgb
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy
warnings.filterwarnings("ignore")

import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Conv2D , SeparableConv2D, MaxPool2D, Flatten , Dropout , BatchNormalization

import tensorflow as tf
from tensorflow import keras

## Data Loading

In [2]:
dfm_ready = pd.read_csv('dfm_ready.csv', index_col=0)
dfm_ready.head().append(dfm_ready.tail())

,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,ARR_DELAY,CRS_ELAPSED_TIME,AIR_TIME,DISTANCE,DAY,FLIGHT_STATUS,OP_CARRIER_Allegiant Air,OP_CARRIER_American Airlines,OP_CARRIER_Delta Airlines,OP_CARRIER_Endeavor Air,OP_CARRIER_Envoy Air,OP_CARRIER_ExpressJet,OP_CARRIER_Frontier Airlines,OP_CARRIER_Hawaiian Airlines,OP_CARRIER_JetBlue Airways,OP_CARRIER_Mesa Airline,OP_CARRIER_PSA Airlines,OP_CARRIER_Republic Airways,OP_CARRIER_SkyWest Airlines,OP_CARRIER_Southwest Airlines,OP_CARRIER_Spirit Airlines,OP_CARRIER_United Airlines,OP_CARRIER_Virgin America,DEST_Atlanta,DEST_Boston,DEST_Charlotte,DEST_Chicago,DEST_Dallas-Fort Worth,DEST_Denver,DEST_Detroit,DEST_Houston,DEST_Las Vegas,DEST_Los Angeles,DEST_Minneapolis,DEST_New York,DEST_Newark,DEST_Orlando,DEST_Philadelphia,DEST_Phoenix,DEST_Salt Lake City,DEST_San Francisco,DEST_Seattle,CRS_DEP_TIME_2,CRS_DEP_TIME_3,CRS_DEP_TIME_4,CRS_ARR_TIME_2,CRS_ARR_TIME_3,CRS_ARR_TIME_4,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12,WEEKDAY_1,WEEKDAY_2,WEEKDAY_3,WEEKDAY_4,WEEKDAY_5,WEEKDAY_6
0,-5.0,15.0,3,3,10.0,-23.0,268.0,225.0,1605.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-8.0,11.0,2,3,7.0,-24.0,99.0,65.0,414.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-5.0,15.0,3,3,5.0,-13.0,134.0,106.0,846.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6.0,19.0,3,3,6.0,-2.0,190.0,157.0,1120.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,-3.0,14.0,2,2,6.0,-16.0,206.0,173.0,1222.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7213439,31.0,12.0,4,4,6.0,9.0,182.0,142.0,1303.0,31,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7213440,-1.0,12.0,3,3,4.0,-23.0,84.0,46.0,331.0,31,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7213441,-4.0,20.0,3,3,7.0,-5.0,100.0,72.0,331.0,31,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7213442,6.0,18.0,4,4,10.0,1.0,181.0,148.0,936.0,31,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7213445,8.0,8.0,3,3,7.0,-4.0,71.0,44.0,130.0,31,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


<b>Conditions for the NN to work properly:</b>

1. Data is purely numerical<br>
2. Data constains no missing values<br>
3. Data is normalized

In [3]:
dfm_ready.drop(['TAXI_OUT','WHEELS_OFF','WHEELS_ON','DAY','TAXI_IN','ARR_DELAY'], axis=1, inplace=True)

In [4]:
df = dfm_ready.astype(float)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3982067 entries, 0 to 7213445
Data columns (total 64 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   DEP_DELAY                      float64
 1   CRS_ELAPSED_TIME               float64
 2   AIR_TIME                       float64
 3   DISTANCE                       float64
 4   FLIGHT_STATUS                  float64
 5   OP_CARRIER_Allegiant Air       float64
 6   OP_CARRIER_American Airlines   float64
 7   OP_CARRIER_Delta Airlines      float64
 8   OP_CARRIER_Endeavor Air        float64
 9   OP_CARRIER_Envoy Air           float64
 10  OP_CARRIER_ExpressJet          float64
 11  OP_CARRIER_Frontier Airlines   float64
 12  OP_CARRIER_Hawaiian Airlines   float64
 13  OP_CARRIER_JetBlue Airways     float64
 14  OP_CARRIER_Mesa Airline        float64
 15  OP_CARRIER_PSA Airlines        float64
 16  OP_CARRIER_Republic Airways    float64
 17  OP_CARRIER_SkyWest Airlines    float64
 18  OP

In [6]:
df.isna().sum().sum()

0

In [7]:
y = df['FLIGHT_STATUS']
X = df.drop(['FLIGHT_STATUS', 'DEP_DELAY'], axis = 1)

In [8]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

In [9]:
binarizer = LabelBinarizer()
labels = binarizer.fit_transform(y)

In [10]:
df.shape

(3982067, 64)

In [11]:
def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('LOSS', fontsize=14)
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.show()
    
    plt.figure()
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_accuracy', 'accuracy'])
    plt.title('ACCURACY', fontsize=14)
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.show()

In [12]:
def conf_matrix(conf_mat,):
    
    conf_mat = pd.DataFrame(conf_mat, index = ['0', '1'], columns = ['0', '1'])
    sns.set(font_scale=1.4)
    plt.figure(figsize = (8,7))
    sns.heatmap(conf_mat, cmap= "Blues", linecolor = 'black' , linewidth = 1, annot = True, fmt='')
    plt.xlabel('Predicted', fontsize=14)
    plt.ylabel('True', fontsize=14)
    plt.show()

In [13]:
def model_metrics(a, b):
    
    accuracy = metrics.accuracy_score(a, b)
    precision = precision_score(a, b)
    recall = recall_score(a, b)
    f1 = f1_score(a, b)

    print('Accuracy:', round(accuracy*100, 2),'%')
    print('Precision score:', round(precision*100, 2),'%')
    print('Recall score:', round(recall*100, 2),'%')
    print('F1 score:', round(f1*100, 2),'%')

================================================================================

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

================================================================================

# Modeling

## Model 3

<b>Building the MLP</b>

In [56]:
model_3 = Sequential()

model_3.add(Dense(5, activation='relu', input_shape=(70,)))

model_3.add(Dense(20, activation='relu'))

model_3.add(Dense(1, activation='sigmoid'))

<b>Compiling the Model</b>

In [72]:
model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [73]:
model_3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 5)                 355       
                                                                 
 dense_13 (Dense)            (None, 20)                120       
                                                                 
 dense_14 (Dense)            (None, 1)                 21        
                                                                 
Total params: 496
Trainable params: 496
Non-trainable params: 0
_________________________________________________________________


<b>Fitting the Model</b>

In [84]:
# results3 = model_3.fit(scaled_data, labels, epochs=1, batch_size=10, validation_split=0.2)

================================================================================

================================================================================

### Model 3 with 5 epochs: 

This is an attempt to replicate the previous model because when I did it on a separate notebook with 5 epochs it ended up with val_acc of above 90 and the train_acc of 1.

<b>Building the MLP</b>

In [85]:
model_3a = Sequential()

model_3a.add(Dense(5, activation='relu', input_shape=(64,)))

model_3a.add(Dense(20, activation='relu'))

model_3a.add(Dense(1, activation='sigmoid'))

<b>Compiling the Model</b>

In [86]:
model_3a.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

<b>Fitting the Model</b>

In [87]:
results3a = model_3a.fit(scaled_data, labels, epochs=1, batch_size=10, validation_split=0.2)

318566/318566 [==============================] - 261s 818us/step - loss: 7.8533e-04 - accuracy: 0.9997 - val_loss: 2.9632 - val_accuracy: 0.7503


### Model 3b

In [88]:
model_3b = Sequential()

model_3b.add(Dense(5, activation='relu', input_shape=(64,)))

model_3b.add(Dense(20, activation='relu'))

model_3b.add(Dense(1, activation='sigmoid'))

In [89]:
model_3b.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [90]:
results3b = model_3b.fit(scaled_data, labels, epochs=2, batch_size=10, validation_split=0.2)

Epoch 1/2
318566/318566 [==============================] - 256s 802us/step - loss: 8.2054e-04 - accuracy: 0.9996 - val_loss: 2.8100 - val_accuracy: 0.7130
Epoch 2/2
318566/318566 [==============================] - 258s 809us/step - loss: 5.5481e-12 - accuracy: 1.0000 - val_loss: 2.8811 - val_accuracy: 0.7116


## Model 3c

In [91]:
model_3c = Sequential()

model_3c.add(Dense(20, activation='relu', input_shape=(64,)))
model_3c.add(Dropout(0.2))
model_3c.add(Dense(5, activation='relu'))
model_3c.add(Dropout(0.2))

model_3c.add(Dense(1, activation='sigmoid'))
model_3c.add(Dropout(0.2))

In [92]:
model_3c.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [93]:
results3c = model_3c.fit(scaled_data, labels, epochs=2, batch_size=10, validation_split=0.2)

Epoch 1/2
318566/318566 [==============================] - 280s 876us/step - loss: 1.1546 - accuracy: 0.9232 - val_loss: 1.0069e-06 - val_accuracy: 1.0000
Epoch 2/2
318566/318566 [==============================] - 276s 867us/step - loss: 1.1498 - accuracy: 0.9252 - val_loss: 5.0596e-09 - val_accuracy: 1.0000


## Model 4

In [94]:
model_4 = Sequential()

model_4.add(Dense(20, activation='relu', input_shape=(64,)))
model_4.add(Dropout(0.2))
model_4.add(Dense(5, activation='relu'))
model_4.add(Dropout(0.2))
model_4.add(Dense(1, activation='relu'))
model_4.add(Dropout(0.2))

model_4.add(Dense(1, activation='sigmoid'))
model_4.add(Dropout(0.2))

In [95]:
model_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [96]:
results4 = model_4.fit(scaled_data, labels, epochs=1, batch_size=10, validation_split=0.2)

318566/318566 [==============================] - 296s 926us/step - loss: 1.3411 - accuracy: 0.8636 - val_loss: 0.0576 - val_accuracy: 1.0000


## Model 5

In [97]:
model_5 = Sequential()

model_5.add(Dense(30, activation='relu', input_shape=(64,)))
model_5.add(Dropout(0.2))
model_5.add(Dense(15, activation='relu'))
model_5.add(Dropout(0.2))
model_5.add(Dense(1, activation='relu'))
model_5.add(Dropout(0.2))

model_5.add(Dense(1, activation='sigmoid'))
model_5.add(Dropout(0.2))

In [98]:
model_5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [99]:
results5 = model_5.fit(scaled_data, labels, epochs=5, batch_size=10, validation_split=0.2)

Epoch 1/5
318566/318566 [==============================] - 300s 939us/step - loss: 1.3388 - accuracy: 0.8652 - val_loss: 0.0532 - val_accuracy: 1.0000
Epoch 2/5
318566/318566 [==============================] - 302s 950us/step - loss: 1.3334 - accuracy: 0.8663 - val_loss: 0.0566 - val_accuracy: 1.0000
Epoch 3/5
318566/318566 [==============================] - 300s 941us/step - loss: 1.3347 - accuracy: 0.8663 - val_loss: 0.0553 - val_accuracy: 1.0000
Epoch 4/5
318566/318566 [==============================] - 296s 929us/step - loss: 1.3328 - accuracy: 0.8665 - val_loss: 0.0549 - val_accuracy: 1.0000
Epoch 5/5
318566/318566 [==============================] - 298s 935us/step - loss: 1.3312 - accuracy: 0.8665 - val_loss: 0.0562 - val_accuracy: 1.0000


## Model 6

In [100]:
model_6 = Sequential()

model_6.add(Dense(30, activation='relu', input_shape=(64,)))
model_6.add(Dropout(0.3))
model_6.add(Dense(5, activation='relu'))
model_6.add(Dropout(0.3))


model_6.add(Dense(1, activation='sigmoid'))
model_6.add(Dropout(0.3))

In [101]:
model_6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [102]:
#results6 = model_6.fit(scaled_data, labels, epochs=5, batch_size=10, validation_split=0.2)

## Model 7

In [103]:
model_7 = Sequential()

model_7.add(Dense(16, activation='relu', input_shape=(64,)))
model_7.add(Dropout(0.4))
model_7.add(Dense(8, activation='relu'))
model_7.add(Dropout(0.4))

model_7.add(Dense(1, activation='sigmoid'))

model_7.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [104]:
results7 = model_7.fit(scaled_data, labels, epochs=5, batch_size=10, validation_split=0.2)

Epoch 1/5
318566/318566 [==============================] - 268s 840us/step - loss: 0.0194 - accuracy: 0.9916 - val_loss: 9.5061e-05 - val_accuracy: 1.0000
Epoch 2/5
318566/318566 [==============================] - 267s 840us/step - loss: 0.0162 - accuracy: 0.9932 - val_loss: 3.9498e-06 - val_accuracy: 1.0000
Epoch 3/5
318566/318566 [==============================] - 268s 840us/step - loss: 0.0163 - accuracy: 0.9932 - val_loss: 8.2662e-07 - val_accuracy: 1.0000
Epoch 4/5
318566/318566 [==============================] - 269s 843us/step - loss: 0.0164 - accuracy: 0.9931 - val_loss: 2.4314e-06 - val_accuracy: 1.0000
Epoch 5/5
318566/318566 [==============================] - 270s 846us/step - loss: 0.0162 - accuracy: 0.9932 - val_loss: 1.9689e-06 - val_accuracy: 1.0000


## Model 8

In [18]:
model_8 = Sequential()

model_8.add(Dense(30, activation='relu', input_shape=(64,)))
model_8.add(Dropout(0.4))
model_8.add(Dense(15, activation='relu'))
model_8.add(Dropout(0.4))
model_8.add(Dense(1, activation='relu'))
model_8.add(Dropout(0.4))

model_8.add(Dense(1, activation='sigmoid'))
model_8.add(Dropout(0.4))

model_8.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [106]:
model_8.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [107]:
results8 = model_8.fit(scaled_data, labels, epochs=5, batch_size=10, validation_split=0.2)

Epoch 1/5
318566/318566 [==============================] - 304s 951us/step - loss: 2.5182 - accuracy: 0.7598 - val_loss: 0.0758 - val_accuracy: 1.0000
Epoch 2/5
318566/318566 [==============================] - 301s 943us/step - loss: 2.5195 - accuracy: 0.7623 - val_loss: 0.0755 - val_accuracy: 1.0000
Epoch 3/5
318566/318566 [==============================] - 301s 944us/step - loss: 2.5244 - accuracy: 0.7623 - val_loss: 0.0776 - val_accuracy: 1.0000
Epoch 4/5
318566/318566 [==============================] - 298s 936us/step - loss: 2.5197 - accuracy: 0.7623 - val_loss: 0.0750 - val_accuracy: 1.0000
Epoch 5/5
318566/318566 [==============================] - 298s 935us/step - loss: 2.5219 - accuracy: 0.7622 - val_loss: 0.0766 - val_accuracy: 1.0000


## Model 9

In [15]:
model_9 = Sequential()

model_9.add(Dense(30, activation='relu', input_shape=(64,)))
model_9.add(Dropout(0.3))
model_9.add(Dense(15, activation='relu'))
model_9.add(Dropout(0.3))
model_9.add(Dense(1, activation='relu'))
model_9.add(Dropout(0.3))

model_9.add(Dense(1, activation='sigmoid'))
model_9.add(Dropout(0.3))

model_9.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [16]:
results9 = model_9.fit(scaled_data, labels, epochs=5, batch_size=10, validation_split=0.2)

Epoch 1/5
318566/318566 [==============================] - 324s 1ms/step - loss: 1.9764 - accuracy: 0.7530 - val_loss: 0.2816 - val_accuracy: 0.6288
Epoch 2/5
318566/318566 [==============================] - 309s 970us/step - loss: 1.9680 - accuracy: 0.7564 - val_loss: 0.2860 - val_accuracy: 0.6288
Epoch 3/5
318566/318566 [==============================] - 382s 1ms/step - loss: 1.9665 - accuracy: 0.7567 - val_loss: 0.2780 - val_accuracy: 0.6288
Epoch 4/5
318566/318566 [==============================] - 335s 1ms/step - loss: 1.9656 - accuracy: 0.7566 - val_loss: 0.2812 - val_accuracy: 0.6288
Epoch 5/5
318566/318566 [==============================] - 323s 1ms/step - loss: 1.9707 - accuracy: 0.7564 - val_loss: 0.2812 - val_accuracy: 0.6288


## Model 10

In [ ]:
model_10 = Sequential()

model_10.add(Dense(20, activation='relu'))
model_10.add(Dropout(0.3))
model_10.add(Dense(10, activation='relu'))
model_10.add(Dropout(0.3))
model_10.add(Dense(1, activation='relu'))
model_10.add(Dropout(0.3))

model_10.add(Dense(1, activation='sigmoid'))
model_10.add(Dropout(0.3))

model_10.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results10 = model_10.fit(scaled_data, labels, epochs=5, batch_size=10, validation_split=0.2)

## Model 11

In [ ]:
model_11 = Sequential()

model_11.add(Dense(25, activation='relu', input_shape=(64,)))
model_11.add(Dropout(0.35))
model_11.add(Dense(15, activation='relu'))
model_11.add(Dropout(0.35))
model_11.add(Dense(1, activation='relu'))
model_11.add(Dropout(0.35))

model_11.add(Dense(1, activation='sigmoid'))
model_11.add(Dropout(0.35))

model_11.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results11 = model_11.fit(scaled_data, labels, epochs=5, batch_size=32, validation_split=0.2)

## Model 12

In [ ]:
model_12 = Sequential()

model_12.add(Dense(25, activation='relu', input_shape=(64,)))
model_12.add(Dropout(0.3))
model_12.add(Dense(5, activation='relu'))
model_12.add(Dropout(0.3))
model_12.add(Dense(1, activation='relu'))
model_12.add(Dropout(0.3))

model_12.add(Dense(1, activation='sigmoid'))
model_12.add(Dropout(0.3))

model_12.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results12 = model_12.fit(scaled_data, labels, epochs=5, batch_size=32, validation_split=0.2)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
df.head(3)

In [ ]:
from sklearn import preprocessing

In [ ]:
col_names = list(df.columns)

In [ ]:
mm_scaler = preprocessing.MinMaxScaler()
df_mm = mm_scaler.fit_transform(df)

df_mm = pd.DataFrame(df_mm, columns=col_names)

In [ ]:
y = df_mm['FLIGHT_STATUS']
X = df_mm.drop(['FLIGHT_STATUS', 'DEP_DELAY'], axis = 1)

In [ ]:
df_mm.head(2)

In [ ]:
#scaler = StandardScaler()
#scaled_data = scaler.fit_transform(df_mm)

In [ ]:
binarizer = LabelBinarizer()
labels = binarizer.fit_transform(y)

## Model 13

In [ ]:
model_13 = Sequential()

model_13.add(Dense(30, activation='relu', input_shape=(64,)))
model_13.add(Dropout(0.3))
model_13.add(Dense(15, activation='relu'))
model_13.add(Dropout(0.3))
model_13.add(Dense(1, activation='relu'))
model_13.add(Dropout(0.3))

model_13.add(Dense(1, activation='sigmoid'))
model_13.add(Dropout(0.3))

model_13.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results13 = model_13.fit(df_mm, labels, epochs=5, batch_size=32, validation_split=0.2)

## Model 14

In [ ]:
model_14 = Sequential()

model_14.add(Dense(60, activation='relu', input_shape=(64,)))
model_14.add(Dropout(0.4))
model_14.add(Dense(30, activation='relu'))
model_14.add(Dropout(0.4))
model_14.add(Dense(15, activation='relu'))
model_14.add(Dropout(0.4))

model_14.add(Dense(1, activation='sigmoid'))
model_14.add(Dropout(0.4))

model_14.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results14 = model_14.fit(df_mm, labels, epochs=2, batch_size=32, validation_split=0.1)

## Model 15

In [ ]:
model_15 = Sequential()

model_15.add(Dense(128, activation='relu', input_shape=(64,)))
model_15.add(Dropout(0.4))
model_15.add(Dense(64, activation='relu'))
model_15.add(Dropout(0.4))
model_15.add(Dense(16, activation='relu'))
model_15.add(Dropout(0.4))

model_15.add(Dense(1, activation='sigmoid'))
model_15.add(Dropout(0.4))

model_15.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results15 = model_15.fit(df_mm, labels, epochs=2, batch_size=32, validation_split=0.1)

StandardScaler

In [ ]:
from sklearn import preprocessing

In [ ]:
col_names = list(df.columns)

In [ ]:
s_scaler = preprocessing.StandardScaler()
df_s = s_scaler.fit_transform(df)

df_s = pd.DataFrame(df_s, columns=col_names)

In [ ]:
df_s.head(2)

In [ ]:
y = df_s['FLIGHT_STATUS']
X = df_s.drop(['FLIGHT_STATUS', 'DEP_DELAY'], axis = 1)

In [ ]:
#binarizer = LabelBinarizer()
#labels = binarizer.fit_transform(y)

## Model 16

In [ ]:
model_16 = Sequential()
model_16.add(Dense(20, activation='relu', input_shape=(64,)))
model_16.add(Dropout(0.3))
model_16.add(Dense(5, activation='relu'))
model_16.add(Dropout(0.3))
model_16.add(Dense(1, activation='sigmoid'))
model_16.add(Dropout(0.3))

model_16.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results16 = model_16.fit(df_s, labels, epochs=2, batch_size=32, validation_split=0.3)

## Model 17

In [ ]:
model_17 = Sequential()
model_17.add(Dense(30, activation='relu', input_shape=(64,)))
model_17.add(Dropout(0.3))
model_17.add(Dense(10, activation='relu'))
model_17.add(Dropout(0.3))
model_17.add(Dense(1, activation='relu'))
model_17.add(Dropout(0.3))
model_17.add(Dense(1, activation='sigmoid'))
model_17.add(Dropout(0.3))

model_17.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results17 = model_17.fit(df_s, labels, epochs=2, batch_size=32, validation_split=0.1)

================================================================================

<b>Normalized Dataset</b>

In [ ]:
n_scaler = preprocessing.Normalizer()
df_n = n_scaler.fit_transform(df)

df_n = pd.DataFrame(df_n, columns=col_names)

In [ ]:
df_n.head(2)

In [ ]:
y = df_n['FLIGHT_STATUS']
X = df_n.drop(['FLIGHT_STATUS', 'DEP_DELAY'], axis = 1)

In [ ]:
#binarizer = LabelBinarizer()
#labels = binarizer.fit_transform(y)

## Model 18

In [ ]:
model_18 = Sequential()

model_18.add(Dense(30, activation='relu', input_shape=(64,)))
model_18.add(Dropout(0.3))
model_18.add(Dense(15, activation='relu'))
model_18.add(Dropout(0.3))
model_18.add(Dense(1, activation='relu'))
model_18.add(Dropout(0.3))

model_18.add(Dense(1, activation='sigmoid'))
model_18.add(Dropout(0.3))

model_18.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results18 = model_18.fit(df_n, labels, epochs=5, batch_size=32, validation_split=0.1)

## Model 19

In [ ]:
model_19 = Sequential()

model_19.add(Dense(30, activation='relu', input_shape=(64,)))
model_19.add(Dropout(0.3))
model_19.add(Dense(10, activation='relu'))
model_19.add(Dropout(0.3))
model_19.add(Dense(1, activation='relu'))
model_19.add(Dropout(0.3))

model_19.add(Dense(1, activation='sigmoid'))
model_19.add(Dropout(0.3))

model_19.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results19 = model_19.fit(df_n, labels, epochs=5, batch_size=32, validation_split=0.1)

## Model 20

In [ ]:
model_20 = Sequential()

model_20.add(Dense(30, activation='relu', input_shape=(64,)))
model_20.add(Dropout(0.4))
model_20.add(Dense(15, activation='relu'))
model_20.add(Dropout(0.4))
model_20.add(Dense(5, activation='relu'))
model_20.add(Dropout(0.4))

model_20.add(Dense(1, activation='sigmoid'))
model_20.add(Dropout(0.3))

model_20.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results20 = model_20.fit(df_n, labels, epochs=5, batch_size=32, validation_split=0.1)

## Model 21

In [ ]:
model_21 = Sequential()

model_21.add(Dense(24, activation='relu', input_shape=(64,)))
model_21.add(Dropout(0.3))
model_21.add(Dense(8, activation='relu'))
model_21.add(Dropout(0.3))
model_21.add(Dense(2, activation='relu'))
model_21.add(Dropout(0.3))

model_21.add(Dense(1, activation='sigmoid'))
model_21.add(Dropout(0.3))

model_21.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results21 = model_21.fit(df_n, labels, epochs=5, batch_size=32, validation_split=0.1)